# 동대문 종합시장 scraper

In [7]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bs

from pathlib import Path
from tqdm import tqdm

In [3]:
CUR_PATH = Path('.').resolve()
DATA_PATH = CUR_PATH / 'data'
OUT_PATH = CUR_PATH / 'output'

In [4]:
catcode_list = [
    '10000000', # 원단
    '11000000', # 부자재
    '12000000', # 실
    '13000000', # 레이스
]

In [5]:
url = 'https://www.ddm-mall.com/store/store.php?' # page=4&catcode=13000000

In [6]:
res = requests.get(url)
res

<Response [200]>

In [16]:
res_bs = bs(res.content, 'html.parser')

In [17]:
details = res_bs.find_all('div', {'class': 'storeTable'})

In [19]:
len(details)

21